<a href="https://colab.research.google.com/github/JoaquinGonzalezSimon/Data_science_and_ML_from_Medium/blob/main/220509_Stock_Price_Prediction_using_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This project was taken from the following link and I use it to practice my skills

https://thecleverprogrammer.com/2020/11/14/stock-price-prediction-using-machine-learning/

https://medium.com/coders-camp/180-data-science-and-machine-learning-projects-with-python-6191bc7b9db9

In [4]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 6.4 MB 24.1 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [11]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

import yfinance as yf

from datetime import datetime

from datetime import date, timedelta

In [6]:
def prepare_data(df, forecast_col, forecast_out, test_size):
  label = df[forecast_col].shift(-forecast_out) # new column with last 5 rows in nan
  X = np.array(df[[forecast_col]]) # creating the feature array
  X = preprocessing.scale(X) # Standarizing the feature array
  X_lately = X[-forecast_out:] # Create the columns I want to use later in the prediction model
  X = X[:-forecast_out] # X that will contain the training and testing
  label.dropna(inplace=True) # Dropping NAN values
  y = np.array(label) # Assigning Y

  X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=test_size, random_state=0) #Cross validation, random state at 0 to match exercise

  response = [X_train, X_test, Y_train, Y_test, X_lately]
  return response

In [16]:
d1 = date.today() - timedelta(days=1)
d1 = d1.strftime('%Y-%m-%d')
end_date = d1
end_date

'2022-05-30'

In [17]:
d2 = date.today() - timedelta(days=(365*2))
d2 = d2.strftime('%Y-%m-%d')
start_date = d2
start_date

'2020-05-31'

In [19]:
data = yf.download('GOOG', start=start_date, end=end_date, progress=True)

[*********************100%***********************]  1 of 1 completed


In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 504 entries, 2020-06-01 to 2022-05-27
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       504 non-null    float64
 1   High       504 non-null    float64
 2   Low        504 non-null    float64
 3   Close      504 non-null    float64
 4   Adj Close  504 non-null    float64
 5   Volume     504 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 27.6 KB


In [21]:
data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-06-01,1418.390015,1437.959961,1418.000000,1431.819946,1431.819946,1217100
2020-06-02,1430.550049,1439.609985,1418.829956,1439.219971,1439.219971,1278100
2020-06-03,1438.300049,1446.552002,1429.776978,1436.380005,1436.380005,1256200
2020-06-04,1430.400024,1438.959961,1404.729980,1412.180054,1412.180054,1484300
2020-06-05,1413.170044,1445.050049,1406.000000,1438.390015,1438.390015,1734900


In [22]:
df = data.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 504 entries, 2020-06-01 to 2022-05-27
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       504 non-null    float64
 1   High       504 non-null    float64
 2   Low        504 non-null    float64
 3   Close      504 non-null    float64
 4   Adj Close  504 non-null    float64
 5   Volume     504 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 27.6 KB


In [26]:
df.shape

(504, 6)

In [23]:
forecast_col = 'Close'
forecast_out = 5
test_size = 0.2

In [24]:
X_train, X_test, Y_train, Y_test, X_lately = prepare_data(df, forecast_col, forecast_out, test_size) # pull the function with the mentioned parameters

In [25]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape, X_lately.shape

((399, 1), (100, 1), (399,), (100,), (5, 1))

In [27]:
model = LinearRegression() #initialize the model
model.fit(X_train, Y_train)

LinearRegression()

In [28]:
model.score(X_train, Y_train)

0.9738870153877691

In [29]:
score = model.score(X_test, Y_test) #test the linear regression model
score

0.9728205502981345

In [30]:
forecast = model.predict(X_lately) #set that will contain the forecasted data
forecast

array([2239.85343318, 2127.97930135, 2126.29356   , 2174.16709176,
       2261.92413145])

In [32]:
response = {} #create json object
response['test_score'] = score
response['forecast'] = forecast

print(response)

{'test_score': 0.9728205502981345, 'forecast': array([2239.85343318, 2127.97930135, 2126.29356   , 2174.16709176,
       2261.92413145])}
